# 🎨 AI Creative Platform - Backend on Google Colab

This notebook runs your AI Creative Platform backend on Google Colab with GPU acceleration.

**Made by Rohith Cherukuri**

## 🚀 Features:
- ✅ GPU-accelerated AI processing (10x faster than CPU)
- ✅ ControlNet + Stable Diffusion
- ✅ Public URL via ngrok
- ✅ Connects to your local frontend

## 📋 Instructions:
1. **Enable GPU**: Runtime → Change runtime type → GPU → T4
2. **Run all cells** in order
3. **Copy the ngrok URL** from the last cell
4. **Update your local frontend** to use the Colab backend URL


## 🔧 Setup Environment

In [ ]:
# Check GPU availability
import torch
import subprocess

print("🎮 GPU Check:")
if torch.cuda.is_available():
    print(f"✅ CUDA Available: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("❌ No GPU detected - Make sure to enable GPU in Runtime settings!")

print(f"\n🐍 Python: {torch.version.__version__}")
print(f"🔥 PyTorch: {torch.__version__}")

In [ ]:
# Install dependencies
print("📦 Installing dependencies...")

!pip install -q fastapi==0.109.2
!pip install -q uvicorn[standard]==0.27.1
!pip install -q python-multipart==0.0.9
!pip install -q python-dotenv==1.0.1
!pip install -q Pillow==10.2.0
!pip install -q pydantic==2.6.1
!pip install -q aiofiles==23.2.1

# AI/ML Dependencies - GPU versions
!pip install -q diffusers==0.26.3
!pip install -q transformers==4.38.2
!pip install -q accelerate==0.27.2
!pip install -q controlnet-aux==0.0.10
!pip install -q opencv-python==4.9.0.80
!pip install -q safetensors==0.4.2
!pip install -q huggingface-hub==0.20.3
!pip install -q requests==2.31.0

# Install ngrok for public URL
!pip install -q pyngrok

print("✅ Dependencies installed!")

## 📁 Setup Project Code

In [ ]:
# Create project structure
import os

# Create directories
os.makedirs('backend/routes', exist_ok=True)
os.makedirs('backend/utils', exist_ok=True)
os.makedirs('backend/config', exist_ok=True)
os.makedirs('uploads', exist_ok=True)

print("📁 Project structure created!")

In [ ]:
# Create AI configuration
config_code = '''
import os
from typing import Dict, Any

# Generation parameters
DEFAULT_GENERATION_PARAMS = {
    "num_inference_steps": 15,  # Reduced for faster generation
    "guidance_scale": 7.5,
    "controlnet_conditioning_scale": 1.0,
    "width": 512,
    "height": 512
}

# Quality enhancement prompts
QUALITY_ENHANCERS = [
    "high quality",
    "detailed", 
    "beautiful",
    "masterpiece",
    "best quality"
]

NEGATIVE_PROMPTS = [
    "low quality",
    "blurry",
    "ugly", 
    "bad anatomy",
    "worst quality"
]

# Style-specific enhancements
STYLE_ENHANCEMENTS = {
    "watercolor": [
        "watercolor painting",
        "soft brushstrokes", 
        "artistic",
        "flowing colors"
    ],
    "digital_art": [
        "digital art",
        "vibrant colors",
        "modern",
        "digital painting"
    ],
    "minimalist": [
        "minimalist",
        "clean lines",
        "simple",
        "modern design"
    ]
}

def get_ai_config() -> Dict[str, Any]:
    return {
        "use_ai_models": True,
        "device": "cuda",  # Force GPU on Colab
        "controlnet_model": "lllyasviel/sd-controlnet-canny",
        "stable_diffusion_model": "runwayml/stable-diffusion-v1-5",
        "generation_params": DEFAULT_GENERATION_PARAMS
    }
'''

with open('backend/config/ai_config.py', 'w') as f:
    f.write(config_code)

# Create __init__.py files
with open('backend/config/__init__.py', 'w') as f:
    f.write('# Config package')

with open('backend/utils/__init__.py', 'w') as f:
    f.write('# Utils package')

with open('backend/routes/__init__.py', 'w') as f:
    f.write('# Routes package')

print("⚙️ Configuration created!")

In [ ]:
# Create optimized image processor for Colab
processor_code = '''
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance, ImageOps
import torch
import numpy as np
from typing import Tuple, Optional
import cv2
import os
import logging
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from controlnet_aux import CannyDetector
from config.ai_config import get_ai_config, STYLE_ENHANCEMENTS, QUALITY_ENHANCERS, NEGATIVE_PROMPTS

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ImageProcessor:
    """Colab-optimized image processor with GPU acceleration"""
    
    _instance = None
    _models_loaded = False
    _controlnet_canny = None
    _pipe_canny = None
    _canny_detector = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(ImageProcessor, cls).__new__(cls)
        return cls._instance
    
    def __init__(self):
        if hasattr(self, '_initialized'):
            return
        
        self.config = get_ai_config()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        logger.info(f"🎮 Using device: {self.device}")
        if self.device == "cuda":
            logger.info(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
        
        self.pastel_colors = [
            (230, 230, 250), (255, 182, 193), (176, 224, 230),
            (240, 255, 240), (255, 218, 185), (221, 160, 221), (255, 255, 224)
        ]
        
        self._initialized = True
    
    def _load_controlnet_models(self):
        """Load models with GPU optimization"""
        if ImageProcessor._models_loaded:
            return True
        
        try:
            logger.info("📥 Loading ControlNet models...")
            
            # Load ControlNet
            ImageProcessor._controlnet_canny = ControlNetModel.from_pretrained(
                "lllyasviel/sd-controlnet-canny",
                torch_dtype=torch.float16
            )
            
            # Load pipeline
            ImageProcessor._pipe_canny = StableDiffusionControlNetPipeline.from_pretrained(
                "runwayml/stable-diffusion-v1-5",
                controlnet=ImageProcessor._controlnet_canny,
                torch_dtype=torch.float16,
                safety_checker=None,
                requires_safety_checker=False
            )
            
            ImageProcessor._pipe_canny.to(self.device)
            ImageProcessor._pipe_canny.enable_model_cpu_offload()
            
            # Load preprocessor
            ImageProcessor._canny_detector = CannyDetector()
            
            ImageProcessor._models_loaded = True
            logger.info("✅ Models loaded successfully!")
            return True
            
        except Exception as e:
            logger.error(f"❌ Failed to load models: {e}")
            return False
    
    def apply_style_transformation(self, image: Image.Image, prompt: str, generation_id: str) -> Image.Image:
        """Apply AI transformation with fallback"""
        try:
            if image.mode != 'RGB':
                image = image.convert('RGB')
            
            # Resize for ControlNet
            image = self._resize_for_controlnet(image, 512)
            
            # Try AI generation
            result = self._generate_with_controlnet(image, prompt)
            if result is not None:
                logger.info(f"✅ Generated with AI for {generation_id}")
                return result
            
            # Fallback
            logger.warning("⚠️ Using fallback processing")
            return self._apply_fallback_processing(image, prompt)
            
        except Exception as e:
            logger.error(f"❌ Error: {e}")
            return self._apply_fallback_processing(image, prompt)
    
    def _resize_for_controlnet(self, image: Image.Image, target_size: int = 512) -> Image.Image:
        width, height = image.size
        if width > height:
            new_width = target_size
            new_height = int((height * target_size) / width)
        else:
            new_height = target_size
            new_width = int((width * target_size) / height)
        
        new_width = (new_width // 8) * 8
        new_height = (new_height // 8) * 8
        
        return image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    
    def _generate_with_controlnet(self, image: Image.Image, prompt: str) -> Optional[Image.Image]:
        """Generate with ControlNet"""
        try:
            if not self._load_controlnet_models():
                return None
            
            enhanced_prompt = self._enhance_prompt(prompt)
            control_image = ImageProcessor._canny_detector(image)
            
            logger.info(f"🎨 Generating: {enhanced_prompt[:50]}...")
            
            result = ImageProcessor._pipe_canny(
                prompt=enhanced_prompt,
                image=control_image,
                num_inference_steps=15,  # Faster generation
                guidance_scale=7.5,
                controlnet_conditioning_scale=1.0,
                generator=torch.Generator(device=self.device).manual_seed(42)
            ).images[0]
            
            return result
            
        except Exception as e:
            logger.error(f"❌ ControlNet failed: {e}")
            return None
    
    def _enhance_prompt(self, prompt: str) -> str:
        quality_terms = ", ".join(QUALITY_ENHANCERS)
        return f"{prompt}, {quality_terms}"
    
    def _apply_fallback_processing(self, image: Image.Image, prompt: str) -> Image.Image:
        """Fast fallback processing"""
        # Apply artistic filter
        enhanced = ImageEnhance.Color(image).enhance(1.2)
        softened = enhanced.filter(ImageFilter.GaussianBlur(radius=1))
        return softened
'''

with open('backend/utils/image_processor.py', 'w') as f:
    f.write(processor_code)

print("🖼️ Image processor created!")

In [ ]:
# Create API routes
routes_code = '''
from fastapi import APIRouter, HTTPException
from pydantic import BaseModel
import base64
import io
import os
import uuid
from PIL import Image
from datetime import datetime
from utils.image_processor import ImageProcessor

router = APIRouter()

class DesignRequest(BaseModel):
    prompt: str
    sketch: str

class DesignResponse(BaseModel):
    image_url: str
    generation_id: str
    prompt_used: str
    processing_time: float

@router.post("/generate-design", response_model=DesignResponse)
async def generate_design(request: DesignRequest):
    """Generate design with GPU acceleration"""
    try:
        start_time = datetime.now()
        
        if not request.prompt.strip():
            raise HTTPException(status_code=400, detail="Prompt required")
        
        if not request.sketch:
            raise HTTPException(status_code=400, detail="Sketch required")
        
        # Process sketch
        processor = ImageProcessor()
        
        # Decode image
        try:
            if request.sketch.startswith('data:image'):
                request.sketch = request.sketch.split(',')[1]
            
            image_data = base64.b64decode(request.sketch)
            sketch_image = Image.open(io.BytesIO(image_data))
        except Exception as e:
            raise HTTPException(status_code=400, detail=f"Invalid image: {e}")
        
        # Generate
        generation_id = str(uuid.uuid4())
        processed_image = processor.apply_style_transformation(
            sketch_image, request.prompt, generation_id
        )
        
        # Save
        filename = f"generated_{generation_id}.png"
        filepath = os.path.join("uploads", filename)
        processed_image.save(filepath, "PNG", quality=95)
        
        processing_time = (datetime.now() - start_time).total_seconds()
        
        return DesignResponse(
            image_url=f"/uploads/{filename}",
            generation_id=generation_id,
            prompt_used=request.prompt,
            processing_time=processing_time
        )
        
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Generation failed: {e}")
'''

with open('backend/routes/design.py', 'w') as f:
    f.write(routes_code)

print("🛣️ Routes created!")

In [ ]:
# Create main FastAPI app
main_code = '''
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
import os
from routes.design import router as design_router

app = FastAPI(
    title="AI Creative Platform - Colab Backend",
    description="GPU-accelerated backend running on Google Colab",
    version="1.0.0"
)

# Enable CORS for all origins (Colab setup)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Mount uploads
os.makedirs("uploads", exist_ok=True)
app.mount("/uploads", StaticFiles(directory="uploads"), name="uploads")

# Include routes
app.include_router(design_router)

@app.get("/")
async def root():
    return {
        "message": "🎨 AI Creative Platform - Colab Backend",
        "status": "running",
        "gpu_available": "cuda" in str(os.environ.get("CUDA_VISIBLE_DEVICES", "")),
        "docs": "/docs"
    }

@app.get("/health")
async def health():
    return {"status": "ok", "message": "Colab backend running! 🚀"}
'''

with open('backend/main.py', 'w') as f:
    f.write(main_code)

print("🚀 Main app created!")

## 🌐 Start Server with Public URL

In [ ]:
# Setup ngrok for public URL
from pyngrok import ngrok
import getpass

# Get ngrok auth token (optional but recommended)
print("🔑 Ngrok Setup (Optional but recommended):")
print("1. Go to https://dashboard.ngrok.com/get-started/your-authtoken")
print("2. Copy your authtoken")
print("3. Paste it below (or press Enter to skip)")

auth_token = getpass.getpass("Ngrok authtoken (optional): ")

if auth_token.strip():
    ngrok.set_auth_token(auth_token)
    print("✅ Ngrok authenticated!")
else:
    print("⚠️ Using ngrok without auth (2-hour limit)")

# Kill any existing ngrok tunnels
ngrok.kill()

print("🌐 Ngrok ready!")

In [ ]:
# Start the FastAPI server with ngrok using threading
import uvicorn
import threading
from pyngrok import ngrok
import os
import nest_asyncio

nest_asyncio.apply()

# Ensure we are in the correct directory before changing to backend
os.chdir('/content/')
os.chdir('backend')

# Start ngrok tunnel
try:
    tunnels = ngrok.get_tunnels()
    for tunnel in tunnels:
        if tunnel.public_url.endswith(":8000"):
            print(f"Killing existing ngrok tunnel: {tunnel.public_url}")
            ngrok.disconnect(tunnel.public_url)
except Exception as e:
    print(f"Error checking/killing tunnels: {e}")

public_url = ngrok.connect(8000)
print(f"\n🌍 PUBLIC URL: {public_url}")
print(f"📋 Copy this URL to your local frontend!")
print(f"🔗 API Docs: {public_url}/docs")
print(f"❤️ Health Check: {public_url}/health")

print("\n" + "="*60)
print("🎨 AI CREATIVE PLATFORM - COLAB BACKEND")
print("Made by Rohith Cherukuri")
print("="*60)
print(f"🚀 Server starting on: {public_url}")
print("💡 Update your local frontend to use this URL")
print("🔄 Server will restart if this cell is re-run")
print("="*60 + "\n")

# Function to run the server in a separate thread
def run_server():
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=8000,
        reload=False,
        log_level="info"
    )

# Start server in a daemon thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print("✅ Server is running in the background!")
print("📝 You can continue working while the server runs.")
print("⏹️  To stop the server, interrupt the kernel.")

# Keep the main thread alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped by user")

## 📋 Next Steps

1. **Copy the ngrok URL** from above
2. **Update your local frontend** `.env` file:
   ```
   VITE_API_URL=https://your-ngrok-url.ngrok.io
   ```
3. **Start your local frontend**:
   ```bash
   npm run dev
   ```
4. **Enjoy GPU-accelerated AI generation!** 🚀

## 💡 Tips:
- Keep this Colab tab open while using the app
- GPU generation is ~10x faster than CPU
- Free Colab has usage limits - consider Colab Pro for heavy use
- The ngrok URL changes each time you restart
